<a href="https://colab.research.google.com/github/amphyxs/know-graphs-cw/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import Namespace, NamespaceManager
from rdflib.plugins import sparql
from rdflib.namespace import RDF, RDFS, XSD
from rdflib.serializer import Serializer
from typing import *
import re
import logging

# Настройка логгирования

In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Инициализиация онтологии

In [ ]:
g = Graph()

g.parse('ontology.rdf')
ONTOLOGY_IRI = 'http://www.semanticweb.org/user/ontologies/2024/9/music-knowledge'

def ref(resource: str) -> URIRef:
  return URIRef(f'{ONTOLOGY_IRI}#{resource}')

# Свойства

In [ ]:
belongs_to_country_property = ref('принадлежит_стране')
belongs_to_album_property = ref('принадлежит_альбому')
owns_object_property = ref('владеть')
song_matches_genre_property = ref('соответствовать')
instrument_used_in_song_property = ref('содержать')
mood_created_in_song_property = ref('задавать')

# Python-объекты для классов в онтологии

In [ ]:
class OntologyClass:
  class_name: str
  _class_uri_ref: Optional[URIRef] = None
  uri_ref: URIRef

  @property
  def class_uri_ref(self) -> URIRef:
    if not self._class_uri_ref:
      self._class_uri_ref = ref(self.class_name)

    return self._class_uri_ref

  def __init__(self, name: str, **kwargs: dict) -> None:
    self.uri_ref = ref(self._prepare_name_for_ref(name))
    g.add((self.uri_ref, RDF.type, self.class_uri_ref))

  def _prepare_name_for_ref(self, name: str) -> str:
    name = name.replace(' ', '_')

    return re.sub(r'[^A-Za-z0-9_]', '', name)

In [ ]:
class Artist(OntologyClass):
  class_name = 'Исполнитель'

class Genre(OntologyClass):
  class_name = 'Жанр'
  genres: Dict[str, Self] = dict()

  def __new__(cls, name: str, **kwargs: dict) -> Self:
    for key, value in cls.genres.items(): # Проверка, был ли создан такой жанр
      if name == key:
        return value # Если да, то возвращаем уже созданный инстанс

    instance = super().__new__(cls) # Иначе создаём новый
    cls.genres[name] = instance
    return instance

class Song(OntologyClass):
  class_name = 'Композиция'

  def __init__(self, name: str, owner: Artist, genre: Genre, mood: Mood, instruments: List[MusicalInstrument], **kwargs: dict) -> None:
    super().__init__(name, **kwargs)
    g.add((owner.uri_ref, owns_object_property, self.uri_ref))
    g.add((genre.uri_ref, song_matches_genre_property, self.uri_ref))
    g.add((mood.uri_ref, mood_created_in_song_property, self.uri_ref))

    for instrument in instruments:
        g.add((instrument.uri_ref, instrument_used_in_song_property, self.uri_ref))

class Album(OntologyClass):
  class_name = 'Альбом'

class Country(OntologyClass):
  class_name = 'Страна'

class MusicalInstrument(OntologyClass):
    class_name = 'Музыкальный_инструмент'
    instruments: Dict[str, Self] = dict()

    def __new__(cls, name: str, **kwargs: dict) -> Self:
        for key, value in cls.instruments.items():
            if name == key:
                return value

        instance = super().__new__(cls)
        cls.instruments[name] = instance
        return instance

class Mood(OntologyClass):
    class_name = 'Настроение'
    moods: Dict[str, Self] = dict()

    def __new__(cls, name: str, **kwargs: dict) -> Self:
        for key, value in cls.moods.items():
            if name == key:
                return value

        instance = super().__new__(cls)
        cls.moods[name] = instance
        return instance


# Словарь для определения стран

In [ ]:
COUNTRY_CODES = {
    "USA": "США",
    "GB": "Великобритания",
    "CA": "Канада",
    "DE": "Германия",
    "FR": "Франция",
    "RU": "Россия",
    "AU": "Австралия",
    "JP": "Япония",
    "BR": "Бразилия",
    "IN": "Индия",
}

# Получение инстансов для классов из API

In [ ]:
import requests

API_URL = "https://itunes.apple.com/search"

def fetch_musical_bands(limit: int = 10) -> List[str]:
    url = API_URL
    params = {
        "term": "band",        # General search term to find bands
        "media": "music",
        "entity": "musicArtist",
        "limit": limit,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    return [artist['artistName'] for artist in results]

def fetch_song_data_for_band(band_name: str, limit: int = 10) -> List[Dict[str, str]]:
    url = API_URL
    params = {
        "term": band_name,
        "media": "music",
        "entity": "song",
        "limit": limit,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    return [{'song': song['trackName'], 'genre': song['primaryGenreName'], 'mood': song['mood']} for song in results]

def fetch_album_for_song(song_name: str, artist_name: str) -> str:
    url = API_URL
    params = {
        "term": f"{song_name} {artist_name}",
        "media": "music",
        "entity": "album",
        "limit": 1,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])
    if results:
        return results[0]['collectionName']
    return "Unknown Album"

def fetch_country_for_song(song_name: str, artist_name: str) -> str:
    url = API_URL
    params = {
        "term": f"{song_name} {artist_name}",
        "media": "music",
        "entity": "song",
        "limit": 1,
    }

    # logging.info(f'Запрос к iTunes API для песни: {song_name} исполнителя: {artist_name}')

    response = requests.get(url, params=params)
    # logging.info(f'Ответ от iTunes API: {response.json()}')

    results = response.json().get("results", [])

    if results:
        country_code = results[0].get('country')  # Получаем код страны
        if country_code:
        #     logging.info(f'Код страны для песни "{song_name}" исполнителя "{artist_name}": {country_code}')
            return COUNTRY_CODES.get(country_code, "Неизвестная страна")  # Возвращаем название страны или "Неизвестная страна"
        else:
        #     logging.warning(f'Код страны отсутствует для песни "{song_name}" исполнителя: {artist_name}')
            return "Неизвестная страна"
    # logging.warning(f'Страна не найдена для песни "{song_name}" исполнителя: {artist_name}')
    return "Неизвестная страна"

  def fetch_instruments_for_song(song_name: str, artist_name: str) -> List[str]:
    url = API_URL
    params = {
        "term": f"{song_name} {artist_name}",
        "media": "music",
        "entity": "song",
        "limit": 1,
    }
    response = requests.get(url, params=params)
    results = response.json().get("results", [])

    if results and 'instruments' in results[0]:
        # Предполагаем, что API возвращает данные об инструментах в виде списка в поле 'instruments'
        return results[0]['instruments']

    return []  # Возвращаем пустой список, если информация об инструментах не найдена

In [6]:
import requests


def search_recording(title, artist):
    # Шаг 1: Поиск записи по названию и исполнителю
    url = "https://musicbrainz.org/ws/2/recording/"
    params = {
        'query': f'title:"{title}" AND artist:"{artist}"',
        'fmt': 'json',
        'limit': 1
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['recordings']:
            recording_id = data['recordings'][0]["id"]

            # Шаг 2: Получение деталей о записи
            recording_url = f"https://musicbrainz.org/ws/2/recording/{recording_id}?fmt=json"
            recording_response = requests.get(recording_url)

            if recording_response.status_code == 200:
                recording_details = recording_response.json()
                print(f"Детали записи для {title}:")
                print(f"ID: {recording_id}")
                print("Название:", recording_details.get('title'))
                print("Исполнитель:", ", ".join(
                    [artist['artist']['name'] for artist in recording_details.get('artist-credit', [])]
                ))
                # Ограничимся этой информацией и признаем, что инструменты могут быть недоступны
            else:
                print("Не удалось получить детали записи.")
        else:
            print("Запись не найдена.")
    else:
        print("Ошибка при запросе данных.")

# Используйте функцию для поиска основной информации о записи
search_recording("Bohemian Rhapsody", "Queen")


Детали записи для Bohemian Rhapsody:
ID: ba1a698b-347c-48e8-bdb4-eb5d37df54e7
Название: Bohemian Rhapsody
Исполнитель: 


In [ ]:
bands = fetch_musical_bands()
for band in bands:
    songs_data = fetch_song_data_for_band(band)
    artist_instance = Artist(band)

    for song_data in songs_data:
        album_name = fetch_album_for_song(song_data['song'], band)  # Получаем альбом для песни
        album_instance = Album(album_name)  # Создаем экземпляр альбома
        genre_instance = Genre(song_data["genre"])
        mood_instance = Genre(song_data["mood"])
        song_instance = Song(song_data["song"], artist_instance, genre_instance, mood_instance)
        g.add((song_instance.uri_ref, belongs_to_album_property, album_instance.uri_ref))  # Устанавливаем связь между песней и альбомом

In [ ]:
g.serialize(destination='ontology-from-py.rdf')

<Graph identifier=Nccdc284866e94c938466cd10feb3112f (<class 'rdflib.graph.Graph'>)>